In [27]:
import numpy as np

Examples of shapes:
* Wi -- weight matrix of shape (layers_dims[i], layers_dims[0])
* bi -- bias vector of shape (layers_dims[i], 1)

### Initialization

In [ ]:
def initialization(layer_dims, scaling_factor=1):
    """
    Performs random initialization for the weight matrices and zero initialization for biases.
    layer_dims -- list with the dimensions of each layer in our network
    Returns:
    parameters -- dictionary with parameters
    """
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l - 1]) * scaling_factor
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

    return parameters

In [67]:
def random_initialization(layer_dims):
    return initialization(layer_dims, scaling_factor=0.01)

In [68]:
def xavier_initialization(layer_dims):
    """
    Performs Xavier initialization that adds a scaling factor sqrt(1/prev_layer_dimension)
    """
    return initialization(layer_dims, scaling_factor=np.sqrt(1.0 / layers_dims[l-1]))

In [87]:
def he_initialization(layers_dims):
    """
    Performs He initialization that adds a scaling factor sqrt(2/prev_layer_dimension)
    """  
    return initialization(layer_dims, scaling_factor=np.sqrt(2.0 / layers_dims[l-1]))

### Regularization

In [70]:
def get_L2_regularization_cost(parameters, lambd):
    """
    Return L2 regularization term.
    
    Arguments:
    parameters -- python dictionary containing parameters of the model
    lambd -- regularization hyperparameter, scalar

    Returns:
    cost - value of the regularization term to be used for cost function regularization
    """
    m = Y.shape[1]
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    W3 = parameters["W3"]

    sum_of_squares = np.sum(np.square(W1) + np.sum(np.square(W2)) + np.sum(np.square(W3)))
    L2_regularization_cost = 1/m * lambd/2 * sum_of_squares
    
    return L2_regularization_cost

### Mini-batch

In [71]:
def shuffle(vec, num_examples):
    if len(vec) == 0:
        return vec
    permutation = list(np.random.permutation(num_examples))
    return vec[:, permutation]

In [72]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, shape (input size, number of examples)
    Y -- ground thruth vector, shape (1, number of examples)
    mini_batch_size -- size of the mini-batches, integer
    
    Returns:
    mini_batches -- list of (mini_batch_X, mini_batch_Y)
    """
    
    np.random.seed(seed)
    m = X.shape[1]
    mini_batches = []
        
    shuffled_X = shuffle(X)
    shuffled_Y = shuffle(Y).reshape((1,m))

    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:, k * mini_batch_size : (k+1) * mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k * mini_batch_size : (k+1) * mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        num_remaining_examples = m - mini_batch_size * np.floor(m / mini_batch_size)
        last_example_index = num_complete_minibatches * mini_batch_size + num_remaining_examples
        mini_batch_X = shuffled_X[:, num_complete_minibatches * mini_batch_size : last_example_index]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatches * mini_batch_size : last_example_index]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches


### Optimization algorithms

In [73]:
def relu(z):
    x = np.array(z, copy=True)
    np.maximum(x, 0, x) # modifies x in place
    return x

In [74]:
# Test relu
print ("relu([0, 2]) = " + str(relu(np.array([0,2]))))

relu([0, 2]) = [0 2]


In [75]:
def sigmoid(z):
    return  1.0 / (1 + np.exp(-z))

In [76]:
# Test sigmoid
print("sigmoid([0, 2]) = " + str(sigmoid(np.array([0,2]))))
print("exptected: [ 0.5 0.88079708]") 

sigmoid([0, 2]) = [ 0.5         0.88079708]
exptected: [ 0.5 0.88079708]


In [77]:
def linear_activation_forward(A_prev, W, b, activation):
    """
    Implement the forward propagation for the LINEAR->ACTIVATION layer

    Arguments:
    A_prev -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:
    A -- the output of the activation function, also called the post-activation value 
    cache -- a python dictionary containing "linear_cache" and "activation_cache";
             stored for computing the backward pass efficiently
    """
    
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
    else:
        raise Error("Activation function " + activation + " is not supported.")
    
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return A, cache

In [78]:
def forward_propagation(X, parameters):
    """
    Implement forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID computation
    
    Arguments:
    X -- data, numpy array of shape (input size, number of examples)
    parameters -- output of initialize_parameters_deep()
    
    Returns:
    AL -- last post-activation value
    caches -- list of caches containing:
                every cache of linear_relu_forward() (there are L-1 of them, indexed from 0 to L-2)
                the cache of linear_sigmoid_forward() (there is one, indexed L-1)
    """

    caches = []
    A = X
    L = len(parameters) // 2 # of layers in the network
    
    # Implement [LINEAR -> RELU]*(L-1)
    for l in range(1, L):
        A_prev = A
        j = str(l + 1)
        A, cache = linear_activation_forward(A_prev, parameters["W"+j], parameters["b"+j], "relu")
        caches.append(cache)
    
    # Implement LINEAR -> SIGMOID
    AL, cache = linear_activation_forward(A, parameters["W"+str(L)], parameters["b"+str(L)], "sigmoid")
    caches.append(cache)
    
    assert(AL.shape == (1,X.shape[1]))
            
    return AL, caches

In [79]:
def linear_backward(dZ, cache):
    """
    Implement the linear portion of backward propagation for a single layer (layer l)

    Arguments:
    dZ -- Gradient of the cost with respect to the linear output (of current layer l)
    cache -- tuple of values (A_prev, W, b) coming from the forward propagation in the current layer

    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = 1/m * np.dot(dZ, A_prev.T)
    db = 1/m * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

In [80]:
def linear_activation_backward(dA, cache, activation):
    """
    Implement the backward propagation for the LINEAR->ACTIVATION layer.
    
    Arguments:
    dA -- post-activation gradient for current layer l 
    cache -- tuple of values (linear_cache, activation_cache) we store for computing backward propagation efficiently
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"
    
    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
    else:
        raise Error("Activation function " + activation + " is not supported.")      
    
    dA_prev, dW, db = linear_backward(dZ, linear_cache)

    return dA_prev, dW, db

In [81]:
def backward_propagation(AL, Y, caches):
    """
    Implement the backward propagation for the [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group
    
    Arguments:
    AL -- probability vector, output of the forward propagation
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                every cache of linear_activation_forward() with "relu" (it's caches[l], for l in range(L-1) i.e l = 0...L-2)
                the cache of linear_activation_forward() with "sigmoid" (it's caches[L-1])
    
    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    # Initializing the backpropagation
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    # Lth layer (SIGMOID -> LINEAR) gradients.
    current_cache = caches[L-1]
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = \
        linear_activation_backward(dAL, current_cache, "sigmoid")
    
    for l in reversed(range(L-1)):
        j = str(l + 1)
        # lth layer: (RELU -> LINEAR) gradients.
        current_cache = caches[l]
        grads["dA" + j], grads["dW" + j], grads["db" + j] = \
            linear_activation_backward(grads["dA" + str(L)], current_cache, "relu")

    return grads

In [82]:
def compute_cost(A_last, Y):
    """
    Implement the cost function with L2 regularization. See formula (2) above.
    
    Arguments:
    A_last -- post-activation, output of forward propagation, of shape (output size, number of examples)
    Y -- "true" labels vector, of shape (output size, number of examples)
    
    Returns:
    cost - value of the loss function
    """
    m = Y.shape[1]
    logprobs = np.multiply(np.log(A_last), Y) + np.multiply(np.log(1 - A_last), 1 - Y)
    cost = - 1.0 / m * np.sum(logprobs)
    
    cost = np.squeeze(cost)
    return cost

In [83]:
# Test compute_cost()
A_last = np.array([[0.40682402, 0.01629284, 0.16722898, 0.10118111, 0.40682402]])
Y = np.array([[1, 1, 0, 1, 0]])

assert(compute_cost(A_last, Y) - 1.60250160476 < 10e-5)

In [ ]:
def update_parameters(parameters, grads, learning_rate):
    """
    Update parameters using gradient descent
    """
    L = len(parameters) // 2 # number of layers in the neural network

    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)] 
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]
        
    return parameters

### Neural network model

In [86]:
def nn_model(X, Y, n_h, layers_dims, learning_rate=0.0075, num_iterations=1000, mini_batch_size=64,
             seed=0, print_cost=False):
    """
    Implements a L-layer neural network: [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID.
    """
    np.random.seed(1)
    costs = []

    parameters = random_initialization(layers_dims)

    for i in range(0, num_iterations):
        A_last, cache = forward_propagation(X, parameters)
        cost = compute_cost(A_last, Y)
        grads = backward_propagation(A_last, Y, caches)
        parameters = update_parameters(parameters, grads, learning_rate)
                
        # Print cost
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
            costs.append(cost)

    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

In [ ]:
def predict(train_x, train_y, parameters):
    pass
    return Y_prediction